In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
from socket import socket, AF_INET, SOCK_STREAM
from functools import partial


class LazyConnection:

    def __init__(self, addr, family=AF_INET, type=SOCK_STREAM):
        self.addr = addr
        self.family = family
        self.type = type
        self.sock = None

    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('already connected')

        self.sock = socket(self.family, self.type)
        self.sock.connect(self.addr)
        return self.sock

    def __exit__(self, exc_ty, exc_val, exc_tb):
        self.sock.close()
        self.sock = None


conn = LazyConnection(('www.python.org', 80))
with conn as sock:
    sock.send(b'GET /index.html HTTP/1.1\r\n')
    sock.send(b'Host: www.python.org\r\n')
    sock.send(b'\r\n')
    resp = b''.join(iter(partial(sock.recv, 8192), b''))

print(resp.decode('utf-8'))

HTTP/1.1 301 Moved Permanently
Server: Varnish
Retry-After: 0
Location: https://www.python.org/index.html
Content-Length: 0
Accept-Ranges: bytes
Date: Wed, 13 Jan 2021 08:33:52 GMT
Via: 1.1 varnish
Connection: close
X-Served-By: cache-hkg17930-HKG
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1610526833.547901,VS0,VE0
Strict-Transport-Security: max-age=63072000; includeSubDomains


